# PyMem3DG Tutorial 7 - Unwrapped integrator

It is possible to write an integrator directly in Python such that there is more control over its behavior. It could also be useful if you want to modify certain parameters/data during simulation. Here is a snippet on how to write a simple forward Euler integrator. 

In [ ]:
import pymem3dg as dg
import pymem3dg.boilerplate as dgb
from functools import partial
from pathlib import Path


oblate = "../sample_meshes/oblate.ply"
p = dg.Parameters()

p.bending.Kbc = 8.22e-5

p.tension.form = partial(
    dgb.preferredAreaSurfaceTensionModel,
    modulus=0.1,
    preferredArea=12.4866,
    reservoirArea=0,
)
p.osmotic.form = partial(
    dgb.preferredVolumeOsmoticPressureModel,
    strength=0.02,
    preferredVolume=0.7 * 3.14 * 4 / 3,
    reservoirVolume=0,
)


In [ ]:

geometry = dg.Geometry(inputMesh=oblate)

g = dg.System(geometry=geometry, parameters=p)
g.initialize()

outputDir = Path("output/tutorial7")
outputDir.mkdir(parents=True, exist_ok=True)

fe = dg.Euler(system=g, characteristicTimeStep=0.1,
              tolerance=1e-12, outputDirectory=str(outputDir))

frame = 0
lastSave = g.time
lastProcessMesh = g.time
initTime = g.time
tSave = 2000 * fe.characteristicTimeStep
processMeshPeriod = 30


# Note the constructor without complete data for integration length and save period will have the integrate function disabled
try:
    fe.integrate()
except RuntimeError as e:
    print("Integrate function is deleted!", e)


# We can write the main integration loop in python instead
fe.createMutableNetcdfFile(isContinue=False)
while frame < 10:
    # print(frame)
    # compute forces/energy, and test for fe.EXIT
    fe.status()

    # saveData
    if ((g.time - lastSave > tSave) | (g.time == initTime) | fe.EXIT):
        lastSave = g.time
        fe.saveData(ifOutputTrajFile=True,
                    ifOutputMeshFile=False, ifPrintToConsole=True)
        frame = frame + 1

    if (fe.EXIT):
        break

    # mesh processing
    if (g.time - lastProcessMesh > (processMeshPeriod * fe.timeStep)):
        lastProcessMesh = g.time
        g.mutateMesh(1)
        g.updateConfigurations()

    # time stepping and update variables
    if (g.time == lastProcessMesh):
        g.time = g.time + 1e-10 * fe.characteristicTimeStep
    else:
        fe.march()
        # print(g.time)

fe.closeMutableNetcdfFile()


In [ ]:
# import pymem3dg.visual as dg_vis
# import polyscope as ps
# dg_vis.animate(outputDir+"/traj.nc", meanCurvature = True)
# ps.show()